In [1]:
import sympy as sp
from msu_esd import parallel_single_pass, counter_single_pass, shell_and_tube_one_shell_pass

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 6333 & Homework 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 1
## Given
Water enters a counterflow, double-pipe heat exchanger at a rate of $70\,\frac{kg}{min}$ and is heated from $15^\circ C$ to $60^\circ C$ by an oil with a specific heat of $1.9\,\frac{kJ}{kg\,K}$. The oil enters at $116^\circ C$ and leaves at $27^\circ C$. The overall heat transfer coefficient is $300\,\frac{W}{m^2\,K}$

## Find
a. What heat transfer area is required?
b. What area is required if all conditions remain the same except that a shell and tube heat exchanger is used, with the water making one shell pass and the oil making two tube passes?
c. What exit water temperature would result if, for the exchanger of part (a), the water flow rate were decreased to $50\,\frac{kg}{min}$

## Solution
The specific heat of water will be taken at the average temperature of the water entrance and exit ($C_p=4.18\,\frac{kJ}{kg\,K}$)

### Part A
The oil is the hot fluid and the water is the cold fluid. The condition is unmixed because the fluids never meet.

In [2]:
# Declare constants as given
mc_ = 70
Cp_c_, Cp_h_ = 4.18, 1.9
Tc_in_, Tc_out_ = 15, 60
Th_in_, Th_out_ = 116, 27
U_ = 300

Cc, Ch, Th_out, Th_in, Tc_out, Tc_in = sp.symbols(r'C_c C_h T_{h\,out} T_{h\,in} T_{c\,out} T_{c\,in}')

# Solving for Ch
eq = sp.Eq(Cc*(Tc_out - Tc_in), Ch*(Th_in - Th_out))
eq

Eq(C_c*(-T_{c,in} + T_{c,out}), C_h*(T_{h,in} - T_{h,out}))

In [3]:
Ch_solved = sp.solve(eq, Ch)[0]
Ch_solved

C_c*(-T_{c,in} + T_{c,out})/(T_{h,in} - T_{h,out})

In [4]:
# Solving for Cc
Cc_ = mc_*Cp_c_
Cc_  # kJ per (min deg C)

292.59999999999997

In [5]:
Ch_ = Cc_*(Tc_out_ - Tc_in_)/(Th_in_ - Th_out_)
Ch_  # kJ per (min deg C)

147.94382022471908

In [6]:
C_min_ = min([Ch_, Cc_])
C_min_  # kJ per (min deg C)

147.94382022471908

In [7]:
# Get actual q
q_act_ = Cc_*(Tc_out_ - Tc_in_)
q_act_  # kJ per min

13166.999999999998

In [8]:
# Get q max
q_max_ = C_min_*(Th_in_ - Tc_in_)
q_max_  # kJ per min

14942.325842696626

In [9]:
# Effectiveness
epsilon_ = q_act_/q_max_
epsilon_

0.8811881188118813

In [10]:
# C
C_ = C_min_/Cc_
C_

0.5056179775280899

In [11]:
# Find the ntu value
ntu_ = counter_single_pass(epsilon_, C_)
ntu_

3.11590019646128

The NTU relationship is,

$NTU=\frac{UA}{C_{min}}\rightarrow A=NTU\frac{C_{min}}{U}$

In [12]:
# Finding the area (unit manipulation added)
A_ = ntu_*C_min_/U_*1000/60
A_  # m^2

25.609898805746354

### Part B
Everything is the same except a different correlation is used to determine the $NTU$ value. The correct correlation is equation 2-30 in the book, which is a single shell pass and some multiple of 2 tube passes.

In [13]:
ntu_ = shell_and_tube_one_shell_pass(epsilon_, C_)
ntu_

C:\Users\gmbra\Downloads\Senior First Semester\ESD\Homework 2\msu_esd\ntu_effectiveness.py:61: RuntimeWarning: invalid value encountered in log
  ntu_func = lambda e: np.log((C*e - e*np.sqrt(C**2 + 1.0) + e - 2.0)/(C*e + e*(C**2 + 1.0)**0.5 + e - 2.0))/np.sqrt(


nan

There is no such value of $NTU$, indicating that this scenario is not possible.

### Part C
The area is the same, but the heat transfer could be different. We can solve for $NTU$ first, then get the effectiveness. It is also assumed that the inlet temperature of water and hot fluid temperatures (oil) are the same along with the $C_h$ value for the oil.

In [14]:
# Get the new capacity for the water
Cc_ = Cp_c_*50
Cc_  # kJ per (min*deg C)

209.0

Because the value of $C_c$ is still greater than the value of $C_h$, the heat transfer is the same. The steps for finding the value of $NTU$ and continuing will result in the same $q$.

$q=C_c(T_{out} - T_{in})_c$

In [15]:
# Solve for the water exit
Tc_out_ = q_act_/Cc_ + Tc_in_
Tc_out_  # deg F

78.0

## Answer
a. $A=25.6m^2$
b. Not Possible
c. $T_{c,out}=78^\circ C$

\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 6333 & Homework 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 2
## Given
The attributes of the inlet streams to a heat exchanger are,

| Stream | Inlet Temperature ($^\circ F$) | Capacity ($\frac{Btu}{hr\,^\circ F}$) |
|--------|--------------------------------|---------------------------------------|
| Hot    | 600                            | 50,000                                |
| Cold   | 500                            | 25,000                                |

The UA product in $\frac{Btu}{hr\,^\circ F}$ for the heat exchanger is given by,

$UA=\frac{1}{\frac{0.12}{C_h^{0.8}}+\frac{0.06}{C_c^{0.8}}+2\times10^{-7}}$

where $C_h$ and $C_c$ are the capacities in $\frac{Btu}{hr\,^\circ F}$.

## Find
Using only the $NTU$ method,

a. Find the outlet temperatures and rating for a parallel flow arrangement
b. Find the outlet temperatures and rating for a counter flow arrangement
c. Find the outlet temperatures and rating if two of these heat exchangers are placed in series and are operated in a parallel flow arrangement. What are the interface temperatures?
d. Find the outlet temperatures and rating if two of these heat exchangers are placed in series and operated in a counterflow arrangement. What are the interface temperatures?
e. Based ont he results of the above, discuss the utility of placing parallel and counterflow heat exchangers in series.

## Solution

In [16]:
# Define known constants with units mentioned in the given statement
Cc_ = 25_000
Ch_ = 50_000
Tc_in_ = 500
Th_in_ = 600

# Find the value for UA
# First a symbolic definition
UA = 1/(0.12/Ch**0.8 + 0.06/Cc**0.8 + 2e-7)
UA

1/(0.06/C_c**0.8 + 0.12/C_h**0.8 + 2.0e-7)

In [17]:
UA_ = float(UA.subs([(Cc, Cc_), (Ch, Ch_)]))
UA_  # In Btu per (hr*deg F)

25457.073923275435

In [18]:
# Find NTU
C_min_ = min([Cc_, Ch_])
NTU_ = UA_/C_min_
NTU_

1.0182829569310174

In [19]:
# Get the C_value
C_ = C_min_/max([Cc_, Ch_])
C_

0.5

### Part A
We know the $NTU$ value and $C$, which is enough to solve for the effectiveness.

In [20]:
# Get the effectiveness/rating
# Assuming a one pass parallel
parallel_effectiveness = parallel_single_pass(NTU_, C_, find='e')
parallel_effectiveness

0.5219372748457521

In [21]:
# Find q max
q_max_ = C_min_*(Th_in_ - Tc_in_)
q_max_  # Btu per hr

2500000

In [22]:
# Find the actual q
q_act_A = q_max_*parallel_effectiveness
q_act_A  # In btu per hour

1304843.1871143803

In [23]:
# Find cold outlet temperature
Tc_out_A = q_act_A/Cc_ + Tc_in_
Tc_out_A  # deg F

552.1937274845752

In [24]:
# Find the hot outlet temperature
Th_out_A = Th_in_ - q_act_A/Ch_
Th_out_A  # deg F

573.9031362577124

### Part B
Everything is the same except the effectiveness correlation is different.

In [25]:
# Solving for the effectiveness/rating
counter_effectiveness = counter_single_pass(NTU_, C_, find='e')
counter_effectiveness

0.5703958472801851

In [26]:
q_act_B = q_max_*counter_effectiveness
q_act_B  # Btu per hr

1425989.6182004628

In [27]:
# Find cold outlet temperature
Tc_out_B = q_act_B/Cc_ + Tc_in_
Tc_out_B  # deg F

557.0395847280186

In [28]:
# Find the hot outlet temperature
Th_out_B = Th_in_ - q_act_B/Ch_
Th_out_B  # deg F

571.4802076359907

### Part C

In order to solve this, take the outlet temperatures solved in the previous problems and apply that to the next heat exchanger in the series. The values of $NTU$, $C$, $\xi$, and $UA$ are all the same because the capacity and conductance does not change. The heat transfer is different across each unit.

In [29]:
# From the part A in deg F
Tc_in_ = Tc_out_A
Th_in_ = Th_out_A

q_max_ = C_min_*(Th_in_ - Tc_in_)
q_max_  # Btu per hr

542735.2193284292

In [30]:
q_act_C = parallel_effectiveness*q_max_
q_act_C  # Btu per hr

283273.7413390919

In [31]:
Tc_out_ = q_act_C/Cc_ + Tc_in_
Tc_out_  # deg F

563.5246771381389

In [32]:
Th_out_ = Th_in_ - q_act_C/Ch_
Th_out_  # deg F

568.2376614309305

### Part D
The process is the same as Part C.

In [33]:
# From the part B in deg F
Tc_in_ = Tc_out_B
Th_in_ = Th_out_B

q_max_ = C_min_*(Th_in_ - Tc_in_)
q_max_  # Btu per hr

361015.57269930426

In [34]:
# Get the actual heat across the second unit
q_act_D = q_max_*counter_effectiveness
q_act_D  # Btu per hr

205921.78347116092

In [35]:
Tc_out_ = q_act_D/Cc_ + Tc_in_
Tc_out_  # deg F

565.276456066865

In [36]:
Th_out_ = Th_in_ - q_act_D/Ch_
Th_out_  # deg F

567.3617719665675

### Part E
Based on the above results, two heat exchangers in series does bring the overall outlet temperatures closer together, but it does increase the power required to obtain this heat transfer. If the total power is the sum of the heat transfers across each unit,

In [37]:
# Power change from parallel arrangement
power_A = q_act_A
power_C = q_act_A + q_act_C
(power_C - power_A)/power_A*100  # %

21.709408773137167

In [38]:
# Power change from counter flow arrangement
power_B = q_act_B
power_D = q_act_B + q_act_D
(power_D - power_B)/power_B*100  # %

14.440622907972179

For the parallel flow arrangement, there is an additional 21.7% more power lost to heat when utilizing the two heat exchangers in series. For the counter flow arrangement, there is an additional 14.4% more power lost to heat when placing two heat exchangers in series. Therefore, the counter flow arrangement is better than the parallel arrangement because the counter flow requires less power and brings the two outlet temperatures closer together. This is due to the higher effectiveness value for the counter flow arrangement.

It may also be concluded from this analysis that the addition of another heat exchanger does not necessarily double the heat transfer, meaning that the additional heat exchanger does not provide a substantial impact compared to just the one. It may be more beneficial to increase the effectiveness of just one heat exchanger rather than implementing two heat exchangers. 

## Answer
a. $\xi=0.522$, $T_{c,out}=552.19^\circ F$, $T_{h,out}=573.90^\circ F$
b. $\xi=0.570$, $T_{c,out}=557.04^\circ F$, $T_{h,out}=571.48^\circ F$
c. Effectiveness and temperatures at the interface is the same as Part A. Coming out of the second heat exchanger is: $T_{c,out}=563.52^\circ F$, $T_{h,out}=568.24^\circ F$
d. The effectiveness and temperatures at the interface is the same as Part B. Coming out of the second heat exchanger is: $T_{c,out}=565.28^\circ F$, $T_{h,out}=567.36^\circ F$
e. See above